In [3]:
# libs
import pandas as pd
import glob
import json
import csv
import os
import shutil
import itertools
import sys

In [4]:
#### Global Vars & Flags
data_main = True
if data_main:
    root_path = "E:\Data\Kaggle-Covid\document_parses" # Graham's 
else:
    root_path = "C:\My files\Courses\CIS6050\Data\document_parses" # Abdullah's

paths_csv = True

# temp vars 
source_dir1 = "E:\Data\Kaggle-Covid\document_parses\pdf_json"
source_dir2 = "E:\Data\Kaggle-Covid\document_parses\pmc_json"
dest_dir = "E:\Data\Kaggle-Covid\document_parses\\abstracts_json"

In [ ]:
#### UTILS

# Create json file
def write_to_json(file_in_any, file_out_json):
    with open(file_out_json + ".json", "w") as outfile:
        json.dump(file_in_any, outfile)
        
# Copy files from one directory to another
def copy_file_to_dir(source, destinination):
    for filename in glob.glob(os.path.join(source_dir2, '*.*')):
        shutil.copy(filename, dest_dir) 

# 

In [ ]:
#### NOT RUN UNLESS NECESSARY ####

# creating the unique list of authors (takes like 1M minutes to run to run!!)   
json_dir = glob.glob(f'{root_path}/**/*.json', recursive=True)
authors_list = []
for file in json_dir:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in data["metadata"]["authors"]:
            if item not in authors_list:
                authors_list.append(item)
                
# print(len(authors_list))


In [ ]:
# WIP: authors are matched and text + id is added to json based on the match. Takes forever. Still WIP

id = 0
with open("Paths_JSON_clean.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=",")
    
    # itertools.islice let's us take only a couple items from csv_object item
    for row in itertools.islice(csv_reader, 20):
        
        # there will be missing look ups
        try:
            
            # flattens the iterator list item to string
            filename = (''.join(row))
            abstract_dir = str(root_path) + "/abstracts_json/" + filename  
            print(abstract_dir)
            
            # open up the abstracts json using the filename 
            with open(abstract_dir, 'r', encoding='utf-8') as abstract:
                abstract_data = json.load(abstract)
                
                # checks if an abstract exists
                for text in abstract_data["abstract"]:
                    if text and abstract_data["metadata"]["authors"]:
                        for author in abstract_data["metadata"]["authors"]:
                            #print(len(author))
                            with open("authors_test.json", 'r', encoding='utf-8') as authors:
                                authors_data = json.load(authors)
                                if author in authors_data:
                                    id += 1
                                    loc = authors_data.index(author)  
                                    print(text)
                                    print(id)   
                                    print(loc)
                                    
                                    #print(text)
                                #print(author)
                                #print(type(authors_data))
                        print(type(text))
                    else:
                        continue       
        except:
            
            # print the error message from sys
            print(sys.exc_info()[0])
            
            continue    
